In [12]:
# Mến chào cô!
# Em tên: Trần Huỳnh Chí Nguyên.
# MSSV: 23110136.
# Em xin phép nộp bài tập môn Trí tuệ Nhân tạo ạ!
# Em đặt Scale màn hình là 100% nhằm tránh bị cắt xén giao diện,
    # nên có thể bị khuất nếu thiết bị đang có tỉ lệ khác 100% thưa cô.
# Các bài tập em đều tích lũy đầy đủ trong file này, hy vọng cô xem qua và chăm chế các lỗi sai từ các bài trước của em ạ.
# Em xin đính kèm link gitub ạ:
# Em mến chúc cô sức khỏe!

import tkinter as tk
from tkinter import ttk 
import copy
from collections import deque  
from queue import PriorityQueue
import heapq
import random, math

cars_positions = []
canvas_right_global = None
algorithm_frame = None
algorithm_combobox = None
selected = ""
timer_elapse = 0
timer_running = False
timer_label = None
CELL_SIZE = 45

original_state = [
    [0,0,0,0],
    [0,0,0,0],
    [0,0,0,0],
    [0,0,0,0]
]

goal_state = [
    [0,1,0,0],
    [0,0,1,0],
    [1,0,0,0],
    [0,0,0,1]
]

BOARD_SIZE = len(original_state)

def update_timer():
    global timer_elapse, timer_running, timer_label
    if timer_running:
        timer_elapse+=1
        timer_label.config(text=f"Thời gian: {timer_elapse}s")
        timer_label.after(1000,update_timer)

def reset_game():
    global timer_elapse, timer_running, cars_positions, selected
    timer_elapse = 0
    timer_running = False
    cars_positions = copy.deepcopy(original_state)  
    timer_label.config(text="Thời gian: 0 s")
    draw_board(canvas_right_global, with_cars=False)
    selected = ""

def BFS():
    node = tuple(tuple(row) for row in original_state)  
    if node == tuple(tuple(row) for row in goal_state):  
        return goal_state

    frontier = deque([node]) 
    explored = set()          

    while frontier:
        node = frontier.popleft()   
        explored.add(node)

        a = [list(row) for row in node]  
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                b = [list(row) for row in a]
                if b[i][j] == 1:
                    continue
                b[i][j] = 1

                valid = True
                for r in range(BOARD_SIZE):
                    if sum(b[r]) > 1:
                        valid = False
                        break
                if valid:
                    for c2 in range(BOARD_SIZE):
                        if sum(b[r][c2] for r in range(BOARD_SIZE)) > 1:
                            valid = False
                            break

                if valid:
                    child = tuple(tuple(row) for row in b)
                    if child not in explored and all(child != n for n in frontier):
                        if child == tuple(tuple(row) for row in goal_state):
                            return [list(row) for row in child]
                        frontier.append(child)
    return None

def DFS():
    node = tuple(tuple(row) for row in original_state)  
    if node == tuple(tuple(row) for row in goal_state):  
        return goal_state

    stack = [node]        
    visited = {node}    

    while stack:
        node = stack.pop()    
        a = [list(row) for row in node]

        if a == goal_state:
            return a

        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                b = [list(row) for row in a]
                if b[i][j] == 1:
                    continue
                b[i][j] = 1

                valid = True
                for r in range(BOARD_SIZE):
                    if sum(b[r]) > 1:
                        valid = False
                        break
                if valid:
                    for c2 in range(BOARD_SIZE):
                        if sum(b[r][c2] for r in range(BOARD_SIZE)) > 1:
                            valid = False
                            break

                if valid:
                    state = tuple(tuple(row) for row in b)
                    if state not in visited:
                        visited.add(state)
                        stack.append(state)  
    return None
    
def UCS():
    def distance(p1, p2):
        return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])
    
    original_state_tuple = tuple(tuple(row) for row in original_state)
    que = []
    heapq.heappush(que, (0, original_state_tuple, []))  # (cost, state, positions)
    visited = {}

    while que:
        cost, c, positions = heapq.heappop(que)
        a = [list(row) for row in c]

        if a == goal_state:
            return a

        state_key = tuple(tuple(row) for row in a)
        if state_key in visited and visited[state_key] <= cost:
            continue
        visited[state_key] = cost

        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if a[i][j] == 1:
                    continue
                b = [row.copy() for row in a]
                b[i][j] = 1
                valid = True

                for r in range(BOARD_SIZE):
                    if sum(b[r]) > 1:
                        valid = False
                        break
                for c2 in range(BOARD_SIZE):
                    if sum(b[r][c2] for r in range(BOARD_SIZE)) > 1:
                        valid = False
                        break

                if not valid:
                    continue

                state = tuple(tuple(row) for row in b)

                if not positions:
                    step_cost = distance((0, 0), (i, j))
                else:
                    step_cost = distance(positions[-1], (i, j))

                new_cost = cost + (step_cost if step_cost > 0 else 1)
                new_positions = positions + [(i, j)]

                if state not in visited or new_cost < visited[state]:
                    heapq.heappush(que, (new_cost, state, new_positions))

def DLS(state=None, limit=4):
    if state is None:  
        state = tuple(tuple(row) for row in original_state)

    a = [list(row) for row in state]

    if a == goal_state:
        return a
    elif limit == 0:
        return "cutoff"
    else:
        cutoff_occurred = False
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                b = [row.copy() for row in a]
                if b[i][j] == 1:
                    continue
                b[i][j] = 1

                valid = True
                for r in range(BOARD_SIZE):
                    if sum(b[r]) > 1:
                        valid = False
                        break
                for c2 in range(BOARD_SIZE):
                    if sum(b[r][c2] for r in range(BOARD_SIZE)) > 1:
                        valid = False
                        break

                if not valid:
                    continue

                child = tuple(tuple(row) for row in b)

                result = DLS(child, limit - 1)

                if result == "cutoff":
                    cutoff_occurred = True
                elif result != "failure":
                    return result
                    
        if cutoff_occurred:
            return "cutoff"
        else:
            return "failure"

def IDS(max_depth = BOARD_SIZE):
    for depth in range(max_depth+1):
        result = DLS(limit=depth)
        if result != "cutoff" and result != "failure":
            return result
    return None
    
def GreedySearch():
    def heuristic(p1, goal):
        # Dùng Manhattan distance làm h(n)
        return abs(p1[0] - goal[0]) + abs(p1[1] - goal[1])
    
    original_state_tuple = tuple(tuple(row) for row in original_state)
    que = []
    heapq.heappush(que, (0, original_state_tuple, []))  # (h(n), state, positions)
    visited = set()

    while que:
        h_val, c, positions = heapq.heappop(que)
        a = [list(row) for row in c]

        if a == goal_state:
            return a

        state_key = tuple(tuple(row) for row in a)
        if state_key in visited:
            continue
        visited.add(state_key)

        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if a[i][j] == 1:
                    continue
                b = [row.copy() for row in a]
                b[i][j] = 1
                valid = True

                for r in range(BOARD_SIZE):
                    if sum(b[r]) > 1:
                        valid = False
                        break
                for c2 in range(BOARD_SIZE):
                    if sum(b[r][c2] for r in range(BOARD_SIZE)) > 1:
                        valid = False
                        break

                if not valid:
                    continue

                state = tuple(tuple(row) for row in b)
                new_positions = positions + [(i, j)]

                # h(n): khoảng cách Manhattan từ ô mới (i,j) tới goal queen cuối cùng
                # Giả sử goal cuối là vị trí (BOARD_SIZE-1, BOARD_SIZE-1) (theo goal_state được khai báo ở trên)
                h_n = heuristic((i, j), (BOARD_SIZE-1, BOARD_SIZE-1))
                heapq.heappush(que, (h_n, state, new_positions))
    return None

def A_star():
    def distance(p1, p2):
        # g(n) và h(n) đều dùng Manhattan distance trong ví dụ này
        return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

    original_state_tuple = tuple(tuple(row) for row in original_state)
    que = []
    # (f(n), g(n), state, positions)
    heapq.heappush(que, (0, 0, original_state_tuple, []))
    visited = {}

    while que:
        f_val, g_val, c, positions = heapq.heappop(que)
        a = [list(row) for row in c]

        if a == goal_state:
            return a

        state_key = tuple(tuple(row) for row in a)
        if state_key in visited and visited[state_key] <= g_val:
            continue
        visited[state_key] = g_val

        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if a[i][j] == 1:
                    continue
                b = [row.copy() for row in a]
                b[i][j] = 1
                valid = True

                for r in range(BOARD_SIZE):
                    if sum(b[r]) > 1:
                        valid = False
                        break
                for c2 in range(BOARD_SIZE):
                    if sum(b[r][c2] for r in range(BOARD_SIZE)) > 1:
                        valid = False
                        break

                if not valid:
                    continue

                state = tuple(tuple(row) for row in b)
                new_positions = positions + [(i, j)]

                # g(n): chi phí thật (dùng Manhattan giữa 2 quân liên tiếp)
                if not positions:
                    step_cost = distance((0, 0), (i, j))
                else:
                    step_cost = distance(positions[-1], (i, j))
                g_new = g_val + (step_cost if step_cost > 0 else 1)

                # h(n): ước lượng còn lại → ví dụ khoảng cách Manhattan
                # tới góc dưới phải (tùy định nghĩa goal)
                h_new = distance((i, j), (BOARD_SIZE-1, BOARD_SIZE-1))
                f_new = g_new + h_new

                heapq.heappush(que, (f_new, g_new, state, new_positions))
    return None

def Hill_Climbing(goal_state=goal_state, BOARD_SIZE=BOARD_SIZE):
    def conflicts(state):
        cnt = 0
        # đếm số cặp quân xe tấn công nhau (cùng hàng/cột)
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if state[i][j] == 1:
                    # cùng hàng
                    cnt += sum(state[i][k] for k in range(BOARD_SIZE)) - 1
                    # cùng cột
                    cnt += sum(state[k][j] for k in range(BOARD_SIZE)) - 1
        return cnt // 2  # vì mỗi xung đột đếm 2 lần

    def random_state():
        state = [[0]*BOARD_SIZE for _ in range(BOARD_SIZE)]
        for i in range(BOARD_SIZE):
            j = random.randint(0, BOARD_SIZE-1)
            state[i][j] = 1
        return state

    current = random_state()
    current_cost = conflicts(current)

    while True:
        neighbors = []
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if current[i][j] == 1:
                    for new_j in range(BOARD_SIZE):
                        if new_j != j:
                            new_state = [row[:] for row in current]
                            new_state[i][j] = 0
                            new_state[i][new_j] = 1
                            neighbors.append(new_state)

        # chọn neighbor tốt nhất
        next_state = None
        next_cost = float("inf")
        for n in neighbors:
            c = conflicts(n)
            if c < next_cost:
                next_cost = c
                next_state = n

        if next_cost >= current_cost:  # local minima
            return current

        current = next_state
        current_cost = next_cost

        if current == goal_state or current_cost == 0:
            return current

def Simulated_Annealing(goal_state=goal_state, BOARD_SIZE=BOARD_SIZE, max_iter=100000, T=100, cooling=0.99):
    def conflicts(state):
        cnt = 0
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if state[i][j] == 1:
                    cnt += sum(state[i][k] for k in range(BOARD_SIZE)) - 1
                    cnt += sum(state[k][j] for k in range(BOARD_SIZE)) - 1
        return cnt // 2

    # Khởi tạo: mỗi hàng có đúng 1 quân ở cột ngẫu nhiên
    def random_state():
        state = [[0]*BOARD_SIZE for _ in range(BOARD_SIZE)]
        for i in range(BOARD_SIZE):
            j = random.randint(0, BOARD_SIZE-1)
            state[i][j] = 1
        return state

    # Lân cận: chỉ đổi vị trí quân trong cùng hàng
    def random_neighbor(state):
        neighbor = [row[:] for row in state]
        row = random.randint(0, BOARD_SIZE-1)
        col_old = neighbor[row].index(1)
        col_new = random.randint(0, BOARD_SIZE-1)
        while col_new == col_old:
            col_new = random.randint(0, BOARD_SIZE-1)
        neighbor[row][col_old] = 0
        neighbor[row][col_new] = 1
        return neighbor

    current = random_state()
    current_cost = conflicts(current)

    for step in range(max_iter):
        if current_cost == 0 or (goal_state is not None and current == goal_state):
            return current

        neighbor = random_neighbor(current)
        neighbor_cost = conflicts(neighbor)
        delta = neighbor_cost - current_cost

        if delta < 0 or random.random() < math.exp(-delta / T):
            current, current_cost = neighbor, neighbor_cost

        T *= cooling
        if T < 1e-6:
            break

    return current

def draw_board(canvas,with_cars=False):
    for row in range(BOARD_SIZE):
        for col in range(BOARD_SIZE):
            x1, y1 = col * CELL_SIZE, row * CELL_SIZE
            x2, y2 = x1 + CELL_SIZE, y1 + CELL_SIZE
            color = "white" if ( row + col ) % 2 == 0 else "gray"
            canvas.create_rectangle( x1, y1, x2, y2,
                                     fill = color, outline = "black")
            if with_cars and cars_positions and cars_positions[row][col] == 1:
                canvas.create_text(x1 + CELL_SIZE/2,
                                   y1 + CELL_SIZE/2,
                                   text="♖",
                                   font=("Arial",28),
                                   fill="Red")

def show_algorithm_code(event):
    global algorithm_frame, algorithm_combobox, selected
    for widget in algorithm_frame.winfo_children():
        widget.destroy()
    selected = algorithm_combobox.get()
    if selected == "Uninformed Search   ---   Breadth First Search":
        code_text = """def BFS():
    node = tuple(tuple(row) for row in original_state)  
    if node == tuple(tuple(row) for row in goal_state):  
        return goal_state

    frontier = deque([node]) 
    explored = set()          

    while frontier:
        node = frontier.popleft()   
        explored.add(node)

        a = [list(row) for row in node]  
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                b = [list(row) for row in a]
                if b[i][j] == 1:
                    continue
                b[i][j] = 1

                valid = True
                for r in range(BOARD_SIZE):
                    if sum(b[r]) > 1:
                        valid = False
                        break
                if valid:
                    for c2 in range(BOARD_SIZE):
                        if sum(b[r][c2] for r in range(BOARD_SIZE)) > 1:
                            valid = False
                            break

                if valid:
                    child = tuple(tuple(row) for row in b)
                    if child not in explored and all(child != n for n in frontier):
                        if child == tuple(tuple(row) for row in goal_state):
                            return [list(row) for row in child]
                        frontier.append(child)
    return None"""
        
    elif selected == "Uninformed Search   ---   Depth First Search":
        code_text = """def DFS():
    node = tuple(tuple(row) for row in original_state)  
    if node == tuple(tuple(row) for row in goal_state):  
        return goal_state

    stack = [node]        
    visited = {node}    

    while stack:
        node = stack.pop()    
        a = [list(row) for row in node]

        if a == goal_state:
            return a

        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                b = [list(row) for row in a]
                if b[i][j] == 1:
                    continue
                b[i][j] = 1

                valid = True
                for r in range(BOARD_SIZE):
                    if sum(b[r]) > 1:
                        valid = False
                        break
                if valid:
                    for c2 in range(BOARD_SIZE):
                        if sum(b[r][c2] for r in range(BOARD_SIZE)) > 1:
                            valid = False
                            break

                if valid:
                    state = tuple(tuple(row) for row in b)
                    if state not in visited:
                        visited.add(state)
                        stack.append(state)  
    return None"""
        
    elif selected == "Uninformed Search   ---   Depth Limited Search":
        code_text = """ def DLS(state=None, limit=4):
    if state is None:  
        state = tuple(tuple(row) for row in original_state)

    a = [list(row) for row in state]

    if a == goal_state:
        return a
    elif limit == 0:
        return "cutoff"
    else:
        cutoff_occurred = False
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                b = [row.copy() for row in a]
                if b[i][j] == 1:
                    continue
                b[i][j] = 1

                valid = True
                for r in range(BOARD_SIZE):
                    if sum(b[r]) > 1:
                        valid = False
                        break
                for c2 in range(BOARD_SIZE):
                    if sum(b[r][c2] for r in range(BOARD_SIZE)) > 1:
                        valid = False
                        break

                if not valid:
                    continue

                child = tuple(tuple(row) for row in b)

                result = DLS(child, limit - 1)

                if result == "cutoff":
                    cutoff_occurred = True
                elif result != "failure":
                    return result
                    
        if cutoff_occurred:
            return "cutoff"
        else:
            return "failure" """
        
    elif selected == "Uninformed Search   ---   Uniform Cost Search":
        code_text = """def UCS():
    def distance(p1, p2):
        return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])
    
    original_state_tuple = tuple(tuple(row) for row in original_state)
    que = []
    heapq.heappush(que, (0, original_state_tuple, []))  # (cost, state, positions)
    visited = {}

    while que:
        cost, c, positions = heapq.heappop(que)
        a = [list(row) for row in c]

        if a == goal_state:
            return a

        state_key = tuple(tuple(row) for row in a)
        if state_key in visited and visited[state_key] <= cost:
            continue
        visited[state_key] = cost

        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if a[i][j] == 1:
                    continue
                b = [row.copy() for row in a]
                b[i][j] = 1
                valid = True

                for r in range(BOARD_SIZE):
                    if sum(b[r]) > 1:
                        valid = False
                        break
                for c2 in range(BOARD_SIZE):
                    if sum(b[r][c2] for r in range(BOARD_SIZE)) > 1:
                        valid = False
                        break

                if not valid:
                    continue

                state = tuple(tuple(row) for row in b)

                if not positions:
                    step_cost = distance((0, 0), (i, j))
                else:
                    step_cost = distance(positions[-1], (i, j))

                new_cost = cost + (step_cost if step_cost > 0 else 1)
                new_positions = positions + [(i, j)]

                if state not in visited or new_cost < visited[state]:
                    heapq.heappush(que, (new_cost, state, new_positions))"""
        
    elif selected == "Uninformed Search   ---   Iterative Deepening Search":
        code_text = """def IDS(max_depth = BOARD_SIZE):
    for depth in range(max_depth+1):
        result = DLS(limit=depth)
        if result != "cutoff" and result != "failure":
            return result
    return None"""
        
    elif selected == "Informed Search        ---   Greedy Search":
        code_text = """def GreedySearch():
    def heuristic(p1, goal):
        # Dùng Manhattan distance làm h(n)
        return abs(p1[0] - goal[0]) + abs(p1[1] - goal[1])
    
    original_state_tuple = tuple(tuple(row) for row in original_state)
    que = []
    heapq.heappush(que, (0, original_state_tuple, []))  # (h(n), state, positions)
    visited = set()

    while que:
        h_val, c, positions = heapq.heappop(que)
        a = [list(row) for row in c]

        if a == goal_state:
            return a

        state_key = tuple(tuple(row) for row in a)
        if state_key in visited:
            continue
        visited.add(state_key)

        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if a[i][j] == 1:
                    continue
                b = [row.copy() for row in a]
                b[i][j] = 1
                valid = True

                for r in range(BOARD_SIZE):
                    if sum(b[r]) > 1:
                        valid = False
                        break
                for c2 in range(BOARD_SIZE):
                    if sum(b[r][c2] for r in range(BOARD_SIZE)) > 1:
                        valid = False
                        break

                if not valid:
                    continue

                state = tuple(tuple(row) for row in b)
                new_positions = positions + [(i, j)]

                # h(n): khoảng cách Manhattan từ ô mới (i,j) tới goal queen cuối cùng
                # Giả sử goal cuối là vị trí (BOARD_SIZE-1, BOARD_SIZE-1) (theo goal_state được khai báo ở trên)
                h_n = heuristic((i, j), (BOARD_SIZE-1, BOARD_SIZE-1))
                heapq.heappush(que, (h_n, state, new_positions))
    return None"""

    elif selected == "Informed Search        ---   A_star":
        code_text = """def A_star():
    def distance(p1, p2):
        # g(n) và h(n) đều dùng Manhattan distance trong ví dụ này
        return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

    original_state_tuple = tuple(tuple(row) for row in original_state)
    que = []
    # (f(n), g(n), state, positions)
    heapq.heappush(que, (0, 0, original_state_tuple, []))
    visited = {}

    while que:
        f_val, g_val, c, positions = heapq.heappop(que)
        a = [list(row) for row in c]

        if a == goal_state:
            return a

        state_key = tuple(tuple(row) for row in a)
        if state_key in visited and visited[state_key] <= g_val:
            continue
        visited[state_key] = g_val

        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if a[i][j] == 1:
                    continue
                b = [row.copy() for row in a]
                b[i][j] = 1
                valid = True

                for r in range(BOARD_SIZE):
                    if sum(b[r]) > 1:
                        valid = False
                        break
                for c2 in range(BOARD_SIZE):
                    if sum(b[r][c2] for r in range(BOARD_SIZE)) > 1:
                        valid = False
                        break

                if not valid:
                    continue

                state = tuple(tuple(row) for row in b)
                new_positions = positions + [(i, j)]

                # g(n): chi phí thật (dùng Manhattan giữa 2 quân liên tiếp)
                if not positions:
                    step_cost = distance((0, 0), (i, j))
                else:
                    step_cost = distance(positions[-1], (i, j))
                g_new = g_val + (step_cost if step_cost > 0 else 1)

                # h(n): ước lượng còn lại → ví dụ khoảng cách Manhattan
                # tới góc dưới phải (tùy định nghĩa goal)
                h_new = distance((i, j), (BOARD_SIZE-1, BOARD_SIZE-1))
                f_new = g_new + h_new

                heapq.heappush(que, (f_new, g_new, state, new_positions))
    return None"""

    elif selected == "Local Search               ---   Hill_Climbing":
        code_text = """def Hill_Climbing(goal_state=goal_state, BOARD_SIZE=BOARD_SIZE):
    def conflicts(state):
        cnt = 0
        # đếm số cặp quân xe tấn công nhau (cùng hàng/cột)
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if state[i][j] == 1:
                    # cùng hàng
                    cnt += sum(state[i][k] for k in range(BOARD_SIZE)) - 1
                    # cùng cột
                    cnt += sum(state[k][j] for k in range(BOARD_SIZE)) - 1
        return cnt // 2  # vì mỗi xung đột đếm 2 lần

    def random_state():
        state = [[0]*BOARD_SIZE for _ in range(BOARD_SIZE)]
        for i in range(BOARD_SIZE):
            j = random.randint(0, BOARD_SIZE-1)
            state[i][j] = 1
        return state

    current = random_state()
    current_cost = conflicts(current)

    while True:
        neighbors = []
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if current[i][j] == 1:
                    for new_j in range(BOARD_SIZE):
                        if new_j != j:
                            new_state = [row[:] for row in current]
                            new_state[i][j] = 0
                            new_state[i][new_j] = 1
                            neighbors.append(new_state)

        # chọn neighbor tốt nhất
        next_state = None
        next_cost = float("inf")
        for n in neighbors:
            c = conflicts(n)
            if c < next_cost:
                next_cost = c
                next_state = n

        if next_cost >= current_cost:  # local minima
            return current

        current = next_state
        current_cost = next_cost

        if current == goal_state or current_cost == 0:
            return current"""

    elif selected == "Local Search               ---   Simulated_Annealing":
        code_text = """def Simulated_Annealing(goal_state=goal_state, BOARD_SIZE=BOARD_SIZE, max_iter=100000, T=100, cooling=0.99):
    def conflicts(state):
        cnt = 0
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if state[i][j] == 1:
                    cnt += sum(state[i][k] for k in range(BOARD_SIZE)) - 1
                    cnt += sum(state[k][j] for k in range(BOARD_SIZE)) - 1
        return cnt // 2

    # Khởi tạo: mỗi hàng có đúng 1 quân ở cột ngẫu nhiên
    def random_state():
        state = [[0]*BOARD_SIZE for _ in range(BOARD_SIZE)]
        for i in range(BOARD_SIZE):
            j = random.randint(0, BOARD_SIZE-1)
            state[i][j] = 1
        return state

    # Lân cận: chỉ đổi vị trí quân trong cùng hàng
    def random_neighbor(state):
        neighbor = [row[:] for row in state]
        row = random.randint(0, BOARD_SIZE-1)
        col_old = neighbor[row].index(1)
        col_new = random.randint(0, BOARD_SIZE-1)
        while col_new == col_old:
            col_new = random.randint(0, BOARD_SIZE-1)
        neighbor[row][col_old] = 0
        neighbor[row][col_new] = 1
        return neighbor

    current = random_state()
    current_cost = conflicts(current)

    for step in range(max_iter):
        if current_cost == 0 or (goal_state is not None and current == goal_state):
            return current

        neighbor = random_neighbor(current)
        neighbor_cost = conflicts(neighbor)
        delta = neighbor_cost - current_cost

        if delta < 0 or random.random() < math.exp(-delta / T):
            current, current_cost = neighbor, neighbor_cost

        T *= cooling
        if T < 1e-6:
            break

    return current"""
        
    else:
        code_text = "# Chưa có code"
        
    # Xóa widget cũ
    for widget in algorithm_frame.winfo_children():
        widget.destroy()
    
    # Tạo Text widget với scrollbar
    code_widget = tk.Text(algorithm_frame, font=("Consolas", 12), wrap="none", bg="white")
    code_widget.insert("1.0", code_text)
    code_widget.config(state="disabled")  
    code_widget.pack(side="left", fill="both", expand=True)
    
    scrollbar = tk.Scrollbar(algorithm_frame, command=code_widget.yview)
    scrollbar.pack(side="right", fill="y")
    code_widget.config(yscrollcommand=scrollbar.set)


def start_algo():
    global cars_positions, canvas_right_global, timer_running, timer_elapse
    timer_elapse = 0
    timer_running = True
    update_timer()
    if selected == "Uninformed Search   ---   Breadth First Search":
        cars_positions = BFS()
    elif selected == "Uninformed Search   ---   Depth First Search":
        cars_positions = DFS()
    elif selected == "Uninformed Search   ---   Uniform Cost Search":
        cars_positions = UCS()
    elif selected == "Uninformed Search   ---   Depth Limited Search":
        cars_positions = DLS()
    elif selected == "Uninformed Search   ---   Iterative Deepening Search":
        cars_positions = IDS()
    elif selected == "Informed Search        ---   Greedy Search":
        cars_positions = GreedySearch()
    elif selected == "Informed Search        ---   A_star":
        cars_positions = A_star()
    elif selected == "Local Search               ---   Hill_Climbing":
        cars_positions = Hill_Climbing()
    elif selected == "Local Search               ---   Simulated_Annealing":
        cars_positions = Simulated_Annealing()
    else:
        cars_positions = None
        
    draw_board(canvas_right_global, with_cars=True)
    timer_running = False  

def main():
    root=tk.Tk()
    root.title("Bài toán N quân xe")
    root.configure(bg="#ecf0f1")
    root.state("zoomed")
    root.resizable(False,False)

    title_label = tk.Label(root,
                           text="BÀI TOÁN N QUÂN XE",
                           font=("Arial",35,"bold"),
                           bg="#ecf0f1",
                           fg="#2c3e50")
    title_label.pack(side="top",pady=20) 

    # Left frame
    left_frame = tk.Frame(root, bg="#34495e")
    left_frame.place(relx=0.0, rely=0.0, anchor="nw",
                        x=80, y=180, width=800, height=830)
    
    global algorithm_combobox, algorithm_frame
    algorithm_combobox= ttk.Combobox(left_frame, 
                                  values=["Uninformed Search   ---   Breadth First Search", 
                                          "Uninformed Search   ---   Depth First Search", 
                                          "Uninformed Search   ---   Uniform Cost Search", 
                                          "Uninformed Search   ---   Depth Limited Search",
                                          "Uninformed Search   ---   Iterative Deepening Search", 
                                          "Informed Search        ---   Greedy Search",
                                          "Informed Search        ---   A_star",
                                          "Local Search               ---   Hill_Climbing",
                                          "Local Search               ---   Simulated_Annealing"],
                                  font=("Arial", 13))
    algorithm_combobox.current(0)  
    algorithm_combobox.pack(side="top", fill="x", padx=10, pady=10)
    algorithm_combobox.bind("<<ComboboxSelected>>", show_algorithm_code)

    algorithm_frame = tk.Frame(left_frame, bg="white")
    algorithm_frame.pack(fill="both", expand=True, padx=15, pady=20)
    
    # Control frame
    control_frame = tk.Frame(root, bg="#34495e")
    control_frame.place(relx=1.0, rely=1.0, anchor="se",
                        y=-BOARD_SIZE*CELL_SIZE-120,
                        x=-BOARD_SIZE*CELL_SIZE+100)

    global timer_label
    timer_label = tk.Label(control_frame, text="Thời gian: 0 s",
                           font=("Arial",18))
    timer_label.grid(row=0, column=1, columnspan=4, pady=15)

    btn_start = tk.Button(control_frame, text="Bắt đầu", font=("Arial",18), command=start_algo)
    btn_start.grid(row=1, column=1, padx=15, pady=15)

    btn_reset = tk.Button(control_frame, text="Chơi lại", font=("Arial",18), command=reset_game)
    btn_reset.grid(row=1, column=3, padx=15, pady=15)
    
    # Board frame
    frame = tk.Frame(root, bg="black")
    frame.place(relx=1.0, rely=1.0, anchor="se", y=-80, x=-80)

    canvas_left = tk.Canvas(frame,
                            width= BOARD_SIZE * CELL_SIZE,
                            height= BOARD_SIZE * CELL_SIZE)
    canvas_left.grid(row=0, column=0, padx=5, pady=5)
    draw_board(canvas_left, with_cars=False)

    canvas_right = tk.Canvas(frame,
                             width = BOARD_SIZE * CELL_SIZE,
                             height = BOARD_SIZE * CELL_SIZE)
    canvas_right.grid(row=0, column=1, padx=5, pady=5)
    global canvas_right_global
    canvas_right_global = canvas_right
    draw_board(canvas_right_global, with_cars=False)

    root.mainloop()
    
if __name__ == "__main__":
    main()
